# Day 3 - Conversational AI - aka Chatbot by using gr.ChatInterface!

In [3]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCV


In [5]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [11]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

# Student Octavio O. has pointed out that this isn't quite as straightforward for Claude -
# see the excellent contribution in community-contributions "Gradio_issue_with_Claude" that handles Claude.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # print("\nHistory is:")
    # for hist in history:
    #     print(f"\t{hist}")
    # history 是由 Gradio 的 ChatInterface 自動管理並傳入 chat() 函數的。
    # 它包含了使用者和 AI 助手的整個對話紀錄。
    # 每次新的訊息送出時，這些歷史會被一併送進 OpenAI 的模型，確保回覆有上下文可參考。
    
    print("\nMessages is:")
    for message in messages:
        print(f"\t{message}")

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## And then enter Gradio's magic!

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.



Messages is:
	{'role': 'system', 'content': 'You are a helpful assistant'}
	{'role': 'user', 'content': 'Hi, How are you today?'}

Messages is:
	{'role': 'system', 'content': 'You are a helpful assistant'}
	{'role': 'user', 'metadata': None, 'content': 'Hi, How are you today?', 'options': None}
	{'role': 'assistant', 'metadata': None, 'content': "Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?", 'options': None}
	{'role': 'user', 'content': 'What are recommended places to visit in Taipei (in Chinese)?'}

Messages is:
	{'role': 'system', 'content': 'You are a helpful assistant'}
	{'role': 'user', 'metadata': None, 'content': 'Hi, How are you today?', 'options': None}
	{'role': 'assistant', 'metadata': None, 'content': "Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you with whatever you need. How can I assist you today?", 'options': None}
	{'role': 'user', 'm

In [13]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [14]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


In [16]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7896

To create a public link, set `share=True` in `launch()`.


In [18]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

In [13]:
import gradio as gr
import openai

# openai.api_key = "your-openai-key"  # 請填入你的 OpenAI API Key

# 根據模型動態產生 system prompt
def get_system_message(model_name):
    return f"You are a helpful assistant powered by {model_name}."

# chat 主邏輯：Streaming 模式 + 回傳 chatbot 和清空 textbox
def chat(message, history, model_name):
    system_message = {"role": "system", "content": get_system_message(model_name)}
    messages = [system_message] + history + [{"role": "user", "content": message}]

    print(f"\nMessages length is : {len(messages)}, and Model name is : {model_name}")
    if len(messages) >= 2:
        print(f"The second to last message is: {messages[-2]}")
        print(f"The last message is: {messages[-1]}")
    elif messages:
        print(f"The only message is: {messages[-1]}")
    else:
        print("There are no messages.")
        

    # 使用 OpenAI 的 streaming 回應
    stream = openai.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ''
        response += delta
        yield (
            history + [
                {"role": "user", "content": message},
                {"role": "assistant", "content": response}
            ],
            ""  # ✅ 第二個值是為了清空 Textbox
        )

# 清除對話與狀態
def clear():
    return [], []

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 多模型 Streaming 聊天機器人 (OpenAI)")

    model_selector = gr.Dropdown(
        label="選擇模型",
        choices=["gpt-3.5-turbo", "gpt-4"],
        value="gpt-3.5-turbo"
    )

    chatbot = gr.Chatbot(label="聊天記錄", type="messages")
    textbox = gr.Textbox(label="輸入你的訊息", placeholder="例如：請問今天的天氣如何？")
    history_state = gr.State([])

    # Textbox 提交事件：chat() 回傳 chatbot 和 textbox（清空）
    textbox.submit(
        fn=chat,
        inputs=[textbox, history_state, model_selector],
        outputs=[chatbot, textbox]
    ).then(
        lambda x: x,
        inputs=[chatbot],
        outputs=[history_state]
    )

    # 清除按鈕事件
    gr.Button("🗑️ 清除對話").click(fn=clear, inputs=None, outputs=[chatbot, history_state])

demo.launch()


* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.



Messages length is : 2, and Model name is : gpt-3.5-turbo
The second to last message is: {'role': 'system', 'content': 'You are a helpful assistant powered by gpt-3.5-turbo.'}
The last message is: {'role': 'user', 'content': 'Hi'}

Messages length is : 2, and Model name is : gpt-3.5-turbo
The second to last message is: {'role': 'system', 'content': 'You are a helpful assistant powered by gpt-3.5-turbo.'}
The last message is: {'role': 'user', 'content': 'Hi'}
